# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save top 200m averages of anthropogenic carbon in mumol kg-1 as netcdf files
# requires full model output
# note that some paths are hard-coded throughout the script

In [2]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd

In [3]:
#------
# MODEL INFO
#------

#------
# specify two time periods
#------

year_start1 = 1990
year_end1   = 1999
year_start2 = 2090
year_end2   = 2099

years_list1 = np.arange(year_start1,year_end1+1)
years_list2 = np.arange(year_start2,year_end2+1)
num_years1 = year_end1-year_start1+1
num_years2 = year_end2-year_start2+1
print (years_list1)
print (years_list2)
    
#---
# load mesh info
#---

path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
depths   = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)

ind_no_cavity = np.where(cavities==0)[0]
ind_cavities = np.where(cavities==1)[0]


[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999]
[2090 2091 2092 2093 2094 2095 2096 2097 2098 2099]
(72411,)


In [4]:
#-----
# FUNCTIONS
#-----

from numba import njit

@njit
def reorganize_field_in_cavities(ind_cavities,data): 
    for ii in ind_cavities:  # [72408]
        bb = data[:,ii] # get all depth levels at current cavity node
        ind_av = np.where(bb>0)[0] # get indices of all depth levels that are NOT masked
        # if surface value is filled, but thereafter there is a gap: 
        if len(ind_av)>1:
            if (ind_av[1]-ind_av[0])>1:  #any(np.diff(ind_av)>1):  
                bb[ind_av[1]-1]=bb[ind_av[0]] # move "surface" value to correct depth
                bb[ind_av[0]] = 0 # set surface entry to zero
               
        data[:,ii] = bb # overwrite original field
    return data

@njit
def reorganize_field_in_cavities_monthly(ind_cavities,data): 
    for mm in range(0,12):
        #print 'Process month '+str(mm+1)
        for ii in ind_cavities: #ind_cavities: 
            bb = data[ii,:,mm] # get all depth levels at current cavity node
            ind_av = np.where(bb>0)[0] # get indices of all depth levels that are NOT masked
            #ind_av = bb>=0 #bb.mask==False
            #nd_av = np.where(ind_av==True)[0]
            
            # if surface value is filled, but thereafter there is a gap: 
            if (ind_av[1]-ind_av[0])>1:  #any(np.diff(ind_av)>1):  
                bb[ind_av[1]-1]=bb[ind_av[0]] # move "surface" value to correct depth
                bb[ind_av[0]] = 0 # set surface entry to zero
               
            data[ii,:,mm] = bb # overwrite original field
    return data


In [5]:
#-----
# load model data & extract DIC for decade 1
# HERE: add up DIC over depth; 
# in a second step, divide by the correct water thickness at each depth to get avg concentrations
#-----

dz = np.diff(depths)

ind700 = np.argmin(np.abs(depths-700))
ind2000 = np.argmin(np.abs(depths-2000))
#print(ind700,depths[ind700])
#print(ind2000,depths[ind2000])

depth_level = 200
ind_dd = np.where(depths<=depth_level)[0]

# top XX m
dic_mumol_model1a = np.zeros([len(lon),num_years1,12]) # simA, in mumol kg-1
dic_mumol_model1b = np.zeros([len(lon),num_years1,12]) # simD, in mumol kg-1

for yy in range(0,num_years1):
    print ('load year '+str(years_list1[yy])+'...')
    
    #---
    # simA -> rho, DIC
    #---
    print ('simA')
    path = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/rho0_fields/'
    f1    = Dataset(path+'rho0_fesom_'+str(years_list1[yy])+'0101.nc') # DIC
    data_mean1 = f1.variables['rho0'][:,:,:] #.mean(axis=0)  # month x depth x mesh.n2d
    f1.close()
    del path
    
    path = '/pscratch/sd/c/cnissen/COARZE_DIC/simAssp585/'
    f1    = Dataset(path+'bgc02_fesom_'+str(years_list1[yy])+'0101.nc') # DIC
    data_mean1_monthly = f1.variables['bgc02'][:,:,:]  # month x depth x mesh.n2d
    f1.close()
    del path
       
    #----
    # correct data in cavity
    #----
    # set masked values to 0 to get correction within cavity correct
    # (if I don't do that, masked and not-masked values are not correctly recognized with njit)
    data_mean1_monthly[data_mean1_monthly.mask==True]=0 
    # move "surface" value in cavities to correct depth
    for mm in range(0,12):
        data_mean1_monthly[mm,:,:] = reorganize_field_in_cavities(ind_cavities,data_mean1_monthly[mm,:,:])   
    # set zeros back to masked
    data_mean1_monthly = np.ma.masked_where(data_mean1_monthly==0,data_mean1_monthly) 
        
    # loop over depths and add DIC up 
    for dd in ind_dd:#range(0,len(ind_dd)):
        #print dd
        dic = np.transpose(data_mean1_monthly[:,dd,:],[1,0])
        rho = np.transpose(data_mean1[:,dd,:],[1,0])
        dic[dic.mask==True]=0
        rho[rho.mask==True]=0
        dic_mumol_model1a[:,yy,:] = dic_mumol_model1a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
        del dic,rho
    
    
    #---
    # simD -> DIC 
    #---
    print ('simD')
    path = '/pscratch/sd/c/cnissen/COARZE_DIC/simD/'
    f1    = Dataset(path+'bgc02_fesom_'+str(years_list1[yy])+'0101.nc') # DIC
    data_mean2_monthly = f1.variables['bgc02'][:,:,:]  # month x depth x mesh.n2d
    f1.close()
    del path
    
    #----
    # correct data in cavity
    #----
    # set masked values to 0 to get correction within cavity correct
    # (if I don't do that, masked and not-masked values are not correctly recognized with njit)
    data_mean2_monthly[data_mean2_monthly.mask==True]=0 
    # move "surface" value in cavities to correct depth
    for mm in range(0,12):
        data_mean2_monthly[mm,:,:] = reorganize_field_in_cavities(ind_cavities,data_mean2_monthly[mm,:,:])   
    # set zeros back to masked
    data_mean2_monthly = np.ma.masked_where(data_mean2_monthly==0,data_mean2_monthly) 
        
    # loop over depths and add DIC up 
    for dd in ind_dd:#range(0,len(ind_dd)):
        #print dd
        dic = np.transpose(data_mean2_monthly[:,dd,:],[1,0])
        rho = np.transpose(data_mean1[:,dd,:],[1,0])
        dic[dic.mask==True]=0
        rho[rho.mask==True]=0
        dic_mumol_model1b[:,yy,:] = dic_mumol_model1b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
        del dic,rho
        
    del data_mean1,data_mean1_monthly,data_mean2_monthly
    
print('done')


load year 1990...
simA


/tmp/ipykernel_109504/3351868359.py:59: RuntimeWarning: divide by zero encountered in true_divide
  dic_mumol_model1a[:,yy,:] = dic_mumol_model1a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
/tmp/ipykernel_109504/3351868359.py:59: RuntimeWarning: invalid value encountered in true_divide
  dic_mumol_model1a[:,yy,:] = dic_mumol_model1a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]


simD


/tmp/ipykernel_109504/3351868359.py:92: RuntimeWarning: divide by zero encountered in true_divide
  dic_mumol_model1b[:,yy,:] = dic_mumol_model1b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
/tmp/ipykernel_109504/3351868359.py:92: RuntimeWarning: invalid value encountered in true_divide
  dic_mumol_model1b[:,yy,:] = dic_mumol_model1b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]


load year 1991...
simA
simD
load year 1992...
simA
simD
load year 1993...
simA
simD
load year 1994...
simA
simD
load year 1995...
simA
simD
load year 1996...
simA
simD
load year 1997...
simA
simD
load year 1998...
simA
simD
load year 1999...
simA
simD
done


In [6]:
#-----
# load model data & extract DIC for decade 2
# HERE: add up DIC over depth; 
# in a second step, divide by the correct water thickness at each depth to get avg concentrations
#-----

dz = np.diff(depths)

ind700 = np.argmin(np.abs(depths-700))
ind2000 = np.argmin(np.abs(depths-2000))
#print(ind700,depths[ind700])
#print(ind2000,depths[ind2000])

depth_level = 200
ind_dd = np.where(depths<=depth_level)[0]

# top XX m
dic_mumol_model2a = np.zeros([len(lon),num_years2,12]) # simA, in mumol kg-1
dic_mumol_model2b = np.zeros([len(lon),num_years2,12]) # simD, in mumol kg-1

for yy in range(0,num_years2):
    print ('load year '+str(years_list2[yy])+'...')
    
    #---
    # simA -> rho, DIC
    #---
    print ('simA')
    path = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/rho0_fields/'
    f1    = Dataset(path+'rho0_fesom_'+str(years_list2[yy])+'0101.nc') # DIC
    data_mean1 = f1.variables['rho0'][:,:,:] #.mean(axis=0)  # month x depth x mesh.n2d
    f1.close()
    del path
    
    path = '/pscratch/sd/c/cnissen/COARZE_DIC/simAssp585/'
    f1    = Dataset(path+'bgc02_fesom_'+str(years_list2[yy])+'0101.nc') # DIC
    data_mean1_monthly = f1.variables['bgc02'][:,:,:]  # month x depth x mesh.n2d
    f1.close()
    del path
       
    #----
    # correct data in cavity
    #----
    # set masked values to 0 to get correction within cavity correct
    # (if I don't do that, masked and not-masked values are not correctly recognized with njit)
    data_mean1_monthly[data_mean1_monthly.mask==True]=0 
    # move "surface" value in cavities to correct depth
    for mm in range(0,12):
        data_mean1_monthly[mm,:,:] = reorganize_field_in_cavities(ind_cavities,data_mean1_monthly[mm,:,:])   
    # set zeros back to masked
    data_mean1_monthly = np.ma.masked_where(data_mean1_monthly==0,data_mean1_monthly) 
        
    # loop over depths and add DIC up 
    for dd in ind_dd:#range(0,len(ind_dd)):
        #print dd
        dic = np.transpose(data_mean1_monthly[:,dd,:],[1,0])
        rho = np.transpose(data_mean1[:,dd,:],[1,0])
        dic[dic.mask==True]=0
        rho[rho.mask==True]=0
        dic_mumol_model2a[:,yy,:] = dic_mumol_model2a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
        del dic,rho
    
    
    #---
    # simD -> DIC 
    #---
    print ('simD')
    path = '/pscratch/sd/c/cnissen/COARZE_DIC/simD/'
    f1    = Dataset(path+'bgc02_fesom_'+str(years_list2[yy])+'0101.nc') # DIC
    data_mean2_monthly = f1.variables['bgc02'][:,:,:]  # month x depth x mesh.n2d
    f1.close()
    del path
    
    #----
    # correct data in cavity
    #----
    # set masked values to 0 to get correction within cavity correct
    # (if I don't do that, masked and not-masked values are not correctly recognized with njit)
    data_mean2_monthly[data_mean2_monthly.mask==True]=0 
    # move "surface" value in cavities to correct depth
    for mm in range(0,12):
        data_mean2_monthly[mm,:,:] = reorganize_field_in_cavities(ind_cavities,data_mean2_monthly[mm,:,:])   
    # set zeros back to masked
    data_mean2_monthly = np.ma.masked_where(data_mean2_monthly==0,data_mean2_monthly) 
        
    # loop over depths and add DIC up 
    for dd in ind_dd:#range(0,len(ind_dd)):
        #print dd
        dic = np.transpose(data_mean2_monthly[:,dd,:],[1,0])
        rho = np.transpose(data_mean1[:,dd,:],[1,0])
        dic[dic.mask==True]=0
        rho[rho.mask==True]=0
        dic_mumol_model2b[:,yy,:] = dic_mumol_model2b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
        del dic,rho
        
    del data_mean1,data_mean1_monthly,data_mean2_monthly
    
print('done')

load year 2090...
simA


/tmp/ipykernel_109504/909649709.py:59: RuntimeWarning: divide by zero encountered in true_divide
  dic_mumol_model2a[:,yy,:] = dic_mumol_model2a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
/tmp/ipykernel_109504/909649709.py:59: RuntimeWarning: invalid value encountered in true_divide
  dic_mumol_model2a[:,yy,:] = dic_mumol_model2a[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]


simD


/tmp/ipykernel_109504/909649709.py:92: RuntimeWarning: divide by zero encountered in true_divide
  dic_mumol_model2b[:,yy,:] = dic_mumol_model2b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]
/tmp/ipykernel_109504/909649709.py:92: RuntimeWarning: invalid value encountered in true_divide
  dic_mumol_model2b[:,yy,:] = dic_mumol_model2b[:,yy,:] + 1000*np.divide(dic,rho)*dz[dd]


load year 2091...
simA
simD
load year 2092...
simA
simD
load year 2093...
simA
simD
load year 2094...
simA
simD
load year 2095...
simA
simD
load year 2096...
simA
simD
load year 2097...
simA
simD
load year 2098...
simA
simD
load year 2099...
simA
simD
done


In [7]:
#----
# create a 2D field with the correct water depth above the chosen depth_threshold
#----

print ('get water column thickness')

# load one example data array (use rho, which is already corerctly reorganized in cavities)
yy=0
path = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/rho0_fields/'
f1    = Dataset(path+'rho0_fesom_'+str(years_list1[yy])+'0101.nc') # DIC
data_mean1 = f1.variables['rho0'][:,:,:].mean(axis=0)  # month x depth x mesh.n2d
f1.close()
del path
print (data_mean1.shape)
    
water_column_thickness = np.zeros(len(lon))
# loop over nodes
for nn in range(0,len(lon)):
    if np.mod(nn,10000)==0:
        print ('Process node',nn)
    # loop over depths 
    for dd in ind_dd:#range(0,len(ind_dd)):
        rho = data_mean1[dd,nn]
        if rho>1000: # = if rho exists
            #print rho,depths[dd],dz[dd]
            water_column_thickness[nn] = water_column_thickness[nn] + dz[dd]
        del rho
        
print ('Min/Max water thickness with chosen depth level:',np.min(water_column_thickness),np.max(water_column_thickness))

#print 'done'


get water column thickness
(99, 72411)
Process node 0
Process node 10000
Process node 20000
Process node 30000
Process node 40000
Process node 50000
Process node 60000
Process node 70000
Min/Max water thickness with chosen depth level: 0.0 203.5


In [8]:
#-----
# get DIC_anth
# divide DIC fields by water column thickness to get average DIC concentration over chosen depth
#-----

# get anthropogenic DIC: simA - simD
dic_mumol_1990s = dic_mumol_model1a - dic_mumol_model1b
dic_mumol_2090s = dic_mumol_model2a - dic_mumol_model2b
dic_mumol_2090s[dic_mumol_2090s<0]=0 # there is very few locations with negative DIC_anth...
dic_mumol_1990s[dic_mumol_1990s<0]=0 

# get decadal avg
dic_mumol_1990s = np.mean(np.mean(dic_mumol_1990s,axis=2),axis=1)
dic_mumol_2090s = np.mean(np.mean(dic_mumol_2090s,axis=2),axis=1)

# get average DIC_anth concentration
dic_mumol_1990s[water_column_thickness>0] = np.divide(dic_mumol_1990s[water_column_thickness>0],\
                                                      water_column_thickness[water_column_thickness>0])
dic_mumol_2090s[water_column_thickness>0] = np.divide(dic_mumol_2090s[water_column_thickness>0],\
                                                      water_column_thickness[water_column_thickness>0])
dic_mumol_1990s[water_column_thickness==0] = 0
dic_mumol_2090s[water_column_thickness==0] = 0

print ('Min/Max DIC_anth in 1990s with chosen depth level:',np.min(dic_mumol_1990s),np.max(dic_mumol_1990s))
print ('Min/Max DIC_anth in 2090s with chosen depth level:',np.min(dic_mumol_2090s),np.max(dic_mumol_2090s))



Min/Max DIC_anth in 1990s with chosen depth level: 0.0 30.879340504389337
Min/Max DIC_anth in 2090s with chosen depth level: 0.0 289.8234779701577


In [9]:
#------
# save as netcdf
#------

save_to_netcdf = True
if save_to_netcdf:
                
    savepath = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/bottom_fields/'
    source = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_save_topXXm_Canth_mumol.ipynb'
                
    if save_to_netcdf:
        vari_list = ['DIC_anth_1990s_top'+str(depth_level)+'m_mumol_kg',\
                    'DIC_anth_2090s_top'+str(depth_level)+'m_mumol_kg',\
                    'water_column_thickness']
        unit_list = ['mu mol kg-1','mu mol kg-1','m']
        description_list = ['average anthropogenic carbon in 1990s in the top '+\
                            str(depth_level)+'m, unit conversion with surface-referenced potential density',\
                            'average anthropogenic carbon in 1990s in the top '+\
                            str(depth_level)+'m, unit conversion with surface-referenced potential density',\
                            'water column thickness']

        for vv in range(0,len(vari_list)):
            vari = vari_list[vv]
            netcdf_name = vari+'_fesom.nc'

            if not os.path.exists(savepath+netcdf_name):
                print ('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                # create dimension & variable
                w_nc_fid.createDimension('nodes_2d', len(lon))
                w_nc_fid.script    = source
                w_nc_fid.close()
                
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
            try:
                w_nc_var1 = w_nc_fid.createVariable(vari, 'f4',('nodes_2d'))
                w_nc_var1.units = unit_list[vv] 
                w_nc_var1.description = description_list[vv]
            except:
                pass
                            
            if vari in ['DIC_anth_1990s_top'+str(depth_level)+'m_mumol_kg']:
                w_nc_fid.variables[vari][:] = dic_mumol_1990s
            elif vari in ['DIC_anth_2090s_top'+str(depth_level)+'m_mumol_kg']:
                w_nc_fid.variables[vari][:] = dic_mumol_2090s
            elif vari in ['water_column_thickness']:
                w_nc_fid.variables[vari][:] = water_column_thickness
            
            w_nc_fid.close() 
            
            print ('Successfully saved '+vari)

            

Create file /pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/bottom_fields/DIC_anth_1990s_top200m_mumol_kg_fesom.nc
Successfully saved DIC_anth_1990s_top200m_mumol_kg
Create file /pscratch/sd/c/cnissen/HLRN_runs_postprocessed/PAPER2_postprocessed/bottom_fields/DIC_anth_2090s_top200m_mumol_kg_fesom.nc
Successfully saved DIC_anth_2090s_top200m_mumol_kg
Successfully saved water_column_thickness


In [10]:
print(dic_mumol_1990s)

[7.38865828 8.17878104 7.85139586 ... 0.         0.         0.        ]
